In [6]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [7]:
# Model Parameter
model = "resnet34"
model_short = "resnet"
MODEL_MODULE = resnet34
# Dataset parameter
dataset = "tinyimagenet"
dataset_short = "tinyimage"
DATASET_ENUM = DatasetClassN.Tinyimage

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 4
STANDARDIZE = True

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [8]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [9]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [10]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, data_class=DATASET_ENUM, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=STANDARDIZE)

Attack dataset was already established previosly, loading dataset from "pickle/tinyimagenet/resnet34/attack_train.p".


In [11]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 4.4887,  2.3951,  2.3795]],  [[ 4.9561,  4.6104,  4.0827]],  [[ 8.6882,  5.6260,  5.3411]],  [[ 4.8982,  4.1949,  4.0112]],  [[ 5.2832,  4.6097,  3.7944]],  [[ 5.5909,  4.2719,  3.1924]],  [[ 5.1045,  4.1306,  4.0684]],  [[ 4.1887,  3.4991,  3.2849]],  [[ 6.1415,  4.0232,  3.9851]],  [[ 5.2000,  4.7168,  4.4877]],  [[ 5.6874,  4.8412,  4.5542]],  [[ 3.9549,  3.4372,  3.3626]],  [[ 5.0256,  4.7989,  4.2601]],  [[ 8.2368,  7.8002,  6.9966]],  [[ 5.5284,  4.1061,  3.5957]],  [[ 3.4656,  2.5124,  2.3294]],  [[ 3.5968,  3.5340,  3.1534]],  [[ 6.4791,  6.1578,  5.6585]],  [[14.2656, 12.0457, 10.2040]],  [[ 7.3658,  6.8595,  6.0613]],  [[ 5.6968,  4.9079,  4.5961]],  [[ 5.2547,  4.7231,  4.6138]],  [[ 6.1530,  5.9310,  5.6218]],  [[ 3.3779,  3.1769,  3.1135]],  [[ 3.6091,  3.0864,  2.8213]],  [[ 9.9625,  8.0363,  6.9442]],  [[ 5.1751,  4.9264,  4.3970]],  [[ 5.2733,  4.1971,  4.1783]],  [[ 5.8501,  5.3171,  4.5375]],  [[ 3.4073,  2.5898,  2.5213]],  [[ 6.9685,  6.6931,  6.4154]], 

### Attack Model

Simple Model

In [12]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [13]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(203, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32, 16)
        self.bn5 = nn.BatchNorm1d(16)
        self.fc6 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [14]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [15]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE, test_dataset=False, standardize=STANDARDIZE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.3490, 0.7451, 0.2863,  ..., 0.6275, 0.6235, 0.6157], [0.7647, 0.7569, 0.6118,  ..., 0.6196, 0.5804, 0.5882], [0.6824, 0.5569, 0.6980,  ..., 0.6588, 0.6627, 0.6863], ..., [0.6431, 0.6588, 0.5765,  ..., 0.6863, 0.5961, 0.4078], [0.7490, 0.7255, 0.7020,  ..., 0.6000, 0.5804, 0.7961], [0.7098, 0.7333, 0.6941,  ..., 0.5686, 0.7098, 0.5843]],[[0.3490, 0.7451, 0.2863,  ..., 0.6118, 0.6078, 0.6000], [0.7647, 0.7569, 0.6118,  ..., 0.6039, 0.5647, 0.5725], [0.6824, 0.5569, 0.6980,  ..., 0.6431, 0.6353, 0.6588], ..., [0.6471, 0.6627, 0.5804,  ..., 0.6902, 0.6000, 0.4118], [0.7529, 0.7294, 0.7059,  ..., 0.6039, 0.5843, 0.8000], [0.7137, 0.7373, 0.6980,  ..., 0.5725, 0.7137, 0.5882]],[[0.3098, 0.7059, 0.2471,  ..., 0.5765, 0.5647, 0.5569], [0.7255, 0.7176, 0.5725,  ..., 0.5686, 0.5216, 0.5294], [0.6510, 0.5255, 0.6667,  ..., 0.6000, 0.5961, 0.6196], ..., [0.6235, 0.6392,

Attack Evaluation DL Sample

In [16]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[12.2449,  2.7617,  2.5413]],  [[12.0375,  3.2163,  2.8674]],  [[12.4965,  3.8211,  3.8029]],  [[12.0638,  3.9614,  3.5344]],  [[12.3805,  3.4368,  3.1378]],  [[11.7753,  2.9400,  2.5359]],  [[12.1240,  3.7199,  2.7683]],  [[11.9614,  3.7038,  3.2117]],  [[11.4893,  2.8110,  2.4000]],  [[12.0184,  3.5900,  3.3133]],  [[12.5584,  3.8429,  3.7609]],  [[12.7498,  4.2989,  3.2611]],  [[12.3147,  3.5242,  2.7728]],  [[12.1801,  3.3081,  3.2056]],  [[12.8172,  3.5859,  2.8155]],  [[11.8787,  2.9492,  2.5386]],  [[12.5542,  3.1082,  3.0152]],  [[12.6553,  3.3358,  2.7283]],  [[11.9960,  2.9726,  2.9327]],  [[11.6972,  3.0410,  2.7115]],  [[12.6478,  3.9242,  3.6788]],  [[12.5895,  2.7901,  2.5623]],  [[11.9965,  3.1878,  3.1599]],  [[12.1037,  2.8593,  2.6725]],  [[11.8756,  3.5196,  3.3826]],  [[12.3221,  3.7395,  3.5548]],  [[12.0674,  3.1243,  3.0504]],  [[12.1973,  3.0358,  2.9905]],  [[12.5676,  3.6918,  3.3290]],  [[11.7444,  2.8265,  2.2521]],  [[12.2648,  3.1581,  2.8681]], 

### Train Attack model

In [17]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 50000
Batchsize: 64
Inputs:tensor([[[ 6.8564,  6.4503,  6.3610]],  [[ 8.9363,  6.0344,  4.4876]],  [[ 5.0636,  4.9338,  4.5889]],  [[ 8.0812,  6.1837,  5.6837]],  [[ 5.6380,  5.4858,  5.0778]],  [[ 4.3302,  3.7788,  3.7769]],  [[10.4101,  9.8694,  8.0605]],  [[ 3.2727,  2.9887,  2.7587]],  [[ 6.9876,  4.7811,  4.5151]],  [[ 8.7491,  7.9558,  7.0573]],  [[ 9.8509,  8.1678,  8.0188]],  [[ 7.1559,  7.1274,  6.2264]],  [[ 6.3322,  5.4615,  5.0688]],  [[12.5212, 12.1309,  9.3121]],  [[ 4.7487,  4.6423,  4.5539]],  [[ 5.1357,  3.8202,  3.8119]],  [[ 3.7184,  3.1232,  2.7468]],  [[ 6.4182,  6.4107,  4.6118]],  [[ 6.3450,  4.3257,  4.1965]],  [[ 5.2213,  4.5896,  3.7086]],  [[ 6.6234,  6.5072,  6.1691]],  [[ 4.7935,  4.7846,  4.2085]],  [[ 9.1298,  7.4483,  6.5356]],  [[ 4.9716,  4.3721,  3.5949]],  [[ 5.7894,  5.6428,  5.2859]],  [[12.6141, 11.2762,  9.6634]],  [[ 6.8525,  5.0181,  4.7770]], 

### Evaluation

In [18]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 49.56
